# Asymmetric Cryptography

# Prime Theory

Prime numbers are natural numbers greater than 1 that have no positive divisor other than 1 and itself. All positive integers can be represented as product of primes.

Why do we like them in cryptography?
- There are an infinite number of primes (proven by euclid in 300 BCE)
- There is no simple formula for them, they appear irregularly.
- Easy to multiply large primes together, and there is only one correct factor, but it's hard to reverse.

In [2]:
import math

# Works 100% of the time, but slow for large numbers
def is_prime_bruteforce(n: int) -> bool:
    if n <= 1:
        return False
    
    largest_possible_divisor = int(math.sqrt(n)) + 1
    for i in range(2, largest_possible_divisor):
        if n % i == 0:
            return False
    return True

is_prime_bruteforce(19638107451297617283479599890578920365192471493289512735124045587), is_prime_bruteforce(179638107451297617283479599890578920365192471493289512735124045587)

KeyboardInterrupt: 

## Miller Rabin Primality test (MR)

Much faster, but probabilistic.

It performs rounds of testing, each round has a $\frac{1}{4}$ chance of passing desptie being a composite number. This means that after $k$ rounds, the chance is $(\frac{1}{4})^k$. Written another way: $2^{-2*k}$. With 40 rounds, the chances are $2^{-80}$, which is lower than the chance of cosmic rays hitting the RAM while computing it and flipping a bit. In practice, modern cryptographic libraries use $k=40$ or more.

Increasing $k$ scales the computation linearly, but reduces the chances of errors exponentially.

Many libraries also use a combination of:
- Checking low primes brute-force style
- Running a lot of MR rounds
- Testing the primality of $(p-1)/2$ as well

In [3]:
# Miller-rabin primality test

import random

def is_prime_miller_rabin(n: int, k: int = 100) -> bool:
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    # Write n-1 as 2^r * d (with d odd)
    r, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        r += 1

    # Perform k rounds of testing
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)  # a^d mod n

        if x == 1 or x == n - 1:
            continue  # possibly prime, try next round

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break  # passes this round
        else:
            return False  # definitely composite

    return True  # probably prime

is_prime_miller_rabin(179638107451297617283479599890578920365192471493289512735124045587, 5), is_prime_miller_rabin(2**2048 - 1, 40)

(False, False)

In [ ]:
# Generating random prime numbers of a specified bit length

from sympy import nextprime
import secrets

random_number = secrets.randbits(2048)
nextprime(random_number)

12824093748255476097671899486749324483723642520245917471320241258325734628131908449189837842452016652989906112255822166604391296543226524839209023105565620152437534494844882738486386363399606600019580664466228067223013018120557116451560087589109706436680505144155277954504338254524716622273312328611220309599698390286449591155606783203411472346972272051543230062281254743146732767843197931754332283841083319849817951177698018473154488210797195731727425184032655704795320596149079208942809045335297668105990112962952686370889625542210970587616111463500968756793106228306554871460375998072556142232629864242958108270017


## Relatively Prime Numbers

### Greatest Common Divisor

The greatest common divisor (GCD) is the largest posirive integer that divides both $a$ and $b$ without remainder. Two numbers are coprime (relatively prime) if $gcd(a,b)=1$, so they share no common divisors except 1. This does NOT mean they are both prime.

Example: $gcd(8,15)=1$, yet neither are primes.

### Euler's Totient Function ($\varphi$)

$\varphi(n)=$ The number of integers $\le n$ that are coprimes of $n$

Special cases:
- For a prime $p$, $\varphi(p)=(p-1)$
- For a product of primes $n=p*q$, $\varphi(n)=(p-1)(q-1)$

In [19]:
import math

math.gcd(15, 25), math.gcd(15, 28)

(5, 1)

## Modular Arthmetic

Also called "clock arithmetic" where the operation wraps around when reaching a given modulus, similar to how clock hour hand returns to 1 after 12 no matter how many hours pass.

If we add 2 hours to 6, we get 8. If we add 5 hours to 8, we get 1.

$17$ mod $5 = 2$

In cryptography:

$c = m^k$ mod $n$

### Modular Inverse

The modular invesrse of $a$ mod $m$ is a number $x$, such that $a*x \equiv 1$ mod $m$. This only exists if $gcd(a,m)=1$

In [ ]:
# Modular inverse using the pow function
a = 7
p = 13
pow(a, -1, p) # Fermat's little theorem

2

## Fast Modular Exponentiation

Normally, computing $a^b$ mod $n$ would require $b$ number of multiplications and the same number of modulo operations. This is slow for big $b$ numbers.

### Normal exponentiation

$a ^ b$: means having to do multiplication $b$ times.

Example $3^{17} = 3*3*3*3*3*3*3*3*3*3*3*3*3*3*3*3*3$

We know that $a^{b*c} = (a^b)^c$

So we can simplify as: $3 * (3^8)^2 \rightarrow 3 * ((3^4)^2)^2 \rightarrow 3 * (((3^2)^2)^2)^2$ = 5 operations instead of 17. The number of operations is going to be the logarithm of the number $\log{n}$, the same as the number of bits it consists of. 2048 bit number $\rightarrow$ 2048 number of operations.

## Modular exponentiation

This is the same for modular numbers, as

We know that $a^{b*c}$ mod $n = (a^b)^c$ mod $n$


## The Diffie-Hellman Key Exchange

The DH relies on the Discrete Logarithm Problem being hard:

Given $g$, $p$ primes, and $A=g^a$ mod $p$, it's computationally infeasible to find $a$, especially for a large $p$. This generates the multiplicative group of integers modulo $p$: $\mathbb{Z}^*_p$

In [19]:
## Diffie-Hellman Key Exchange with small numbers
g = 3
p = 23

a = 6 # Alice's private key
A = pow(g, a, p) # Alice's public key (A = g^a mod p)
# A = 3^6 mod 23 = 16

b = 15 # Bob's private key
B = pow(g, b, p) # Bob's public key (B = g^b mod p)
# B = 3^15 mod 23 = 12

# Exchange public keys

# Alice computes the shared secret
sa = pow(B, a, p) # s_a = B^a mod p
# sa = 12^6 mod 23 = 9

# Bob computes the shared secret
sb = pow(A, b, p) # s_b = A^b mod p
# sb = 16^15 mod 23 = 9

assert sa == sb
sa, sb

(9, 9)

Public key consists of:
- $g$, generator
- $p$, prime modulus
- $A$ or $B$, public key

$pk_A = (3, 23, 16)$, $pk_B = (3, 23, 12)$

Private key consists of:
- $g$, generator
- $p$, prime modulus
- $a$ or $b$, private key

$sk_A = (3, 23, 6)$, $sk_B = (3, 23, 15)$

$s_{AB} = 9 = s_{BA} = 9$

In [18]:
## Diffie-Hellman Key Exchange with big numbers

import random
import secrets
from sympy import nextprime


# Generator to use, which must be a primitive root modulo p
# Since 5 is a prime, it is fine to use with any prime p (except 5)
g = 5
# Prime number for the size of the group
# 1024 bits is a common size for DHKE
bits = 1024
# Prime number to use as the modulus
# Generate a random prime number of the specified size
p = nextprime(secrets.randbits(bits))

# Generate a private key for alice
a = random.randint(1, p - 1) # Any number below p is fine
A = pow(g, a, p) # Alice's public key

# Generate a private key for bob
b = random.randint(1, p - 1) # Any number below p is fine
B = pow(g, b, p) # Bob's public key

# Calculate the shared secret
sb = pow(A, b, p) # Shared secret for bob
sa = pow(B, a, p) # Shared secret for alice

assert sa == sb
sa, sb

(73150390937866516482388147053604741393865591579446363812163310109385980588192161402150228620977016090672127960489254679917560876397630624338874535075954484744959462015918798809135676554848266198628090877264832916044073826184626212774215004760501810098094454141957060972890174166665096248083388299755264765777,
 73150390937866516482388147053604741393865591579446363812163310109385980588192161402150228620977016090672127960489254679917560876397630624338874535075954484744959462015918798809135676554848266198628090877264832916044073826184626212774215004760501810098094454141957060972890174166665096248083388299755264765777)

In [8]:
alice_key_bytes = sa.to_bytes(bits // 8, byteorder='big')
alice_key_bytes.hex()

'1962e80c5696cae1522ab11bf8683d189906d837a92a6d884e6d131fa45d224b0537a4f12ff2cd1e83bf9045b2f80229db7eee41371b65d14896fbbdd24bea6eb340adfdc074a4b1dbc621b4f922a4ce4c5cf691b1003fae9ae450bd1263fbc0ba2187e2a39341213a9e67762316fe1f2666677cdb4b2a745361307c1be15182'

In [12]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

import hashlib

# Why hash?
# a, key is too long and has variable length (bitsize / 8)
# b, parts of the key is not random enough (first 32 bytes of the key may have common patterns or leading zeros for instance)
key = hashlib.sha256(alice_key_bytes).digest()
message = b'alice sends this message to bob to salute for all the work he has done'

iv = get_random_bytes(16)
cipher_cbc = AES.new(key, AES.MODE_CBC, iv)
ciphertext = cipher_cbc.encrypt(pad(message, 16))
print(ciphertext.hex())

a9dcc886933dc4ff676c30144aa30fe1e4092dba466116a7f792fe0e60f110ef9c8af7a7bbe7b4a23228cb37651cc8a8b0562a5779fc48d4044b626513d9990a38b5d722af5b2d933aa0172f00cb68bd


In [13]:
from Crypto.Util.Padding import unpad

bob_key_bytes = sa.to_bytes(bits // 8, byteorder='big')
key = hashlib.sha256(bob_key_bytes).digest()

cipher = AES.new(key, AES.MODE_CBC, iv)
message = unpad(cipher.decrypt(ciphertext), 16)
message.decode('utf-8')

'alice sends this message to bob to salute for all the work he has done'

In [1]:
# Cryptography library with sensible defaults and safe primitives
# Pycryptodome is a low-level library that is not as safe as cryptography to build protocols manually
!pip install cryptography

  Using cached cffi-1.17.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 46.9 MB/s eta 0:00:00
Using cached cffi-1.17.1-cp313-cp313-macosx_11_0_arm64.whl (178 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Real world example

from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

# 1. Generate shared parameters (use a safe prime group)
parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

# 2. Each party generates their private/public key pair
alice_private_key = parameters.generate_private_key()
bob_private_key = parameters.generate_private_key()

alice_public_key = alice_private_key.public_key()
bob_public_key = bob_private_key.public_key()

# 3. Exchange public keys and compute the shared secret
alice_shared_key = alice_private_key.exchange(bob_public_key)
bob_shared_key = bob_private_key.exchange(alice_public_key)

assert alice_shared_key == bob_shared_key

# 4. Derive a symmetric key from the shared secret using HKDF
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,              # 256-bit key for AES-256
    salt=None,              # Can optionally add a salt
    info=b'handshake data', # Application-specific context (different key for each use-case)
    backend=default_backend()
).derive(alice_shared_key)

print("Shared AES key:", derived_key.hex())


Shared AES key: 5401a4113eb0b7625a8f715dcd971ce34ea48f9836ad473a309c1ac721494064


## Elliptic Curve Diffie-Helman Key Echange

With FFDH, the public key is calculated as: $pk = g^X mod (p)$. Relies on finite field discrete logaritm problem.

With ECDH, it is calculated as: $P = x \cdot G$. Relies on elliptic curve discrete logarithm problem.

Comparison

|Feature|FFDH|ECDH|
|--|--|--|
|Math domain|Integers mod prime $\mathbb{Z_p}$|Points on elliptic curves|
|Operation|$g^a \: mod \: p$|$a \cdot G$|
|Safe key size|Long (2048+ bits)|Shorter (256 bits ≈ 3072-bit DH)|
|Speed|Slow|Fast|

## Example curve

$$
\mathcal{E}: y^2 \equiv x^3 + a*x + 3 \pmod{p}
$$

In [5]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256

# 1. Generate ECC key pairs for Alice and Bob
alice_key = ECC.generate(curve='P-256')  # aka secp256r1
bob_key = ECC.generate(curve='P-256')

# 2. Each party shares their public key
alice_public = alice_key.public_key()
bob_public = bob_key.public_key()

# 3. Each party computes the shared point
alice_shared_point = alice_key.d * bob_public.pointQ
bob_shared_point = bob_key.d * alice_public.pointQ

# 4. Both should be equal
assert alice_shared_point == bob_shared_point

# 5. Extract x-coordinate of shared point to derive symmetric key
shared_x = int(alice_shared_point.x).to_bytes(32, 'big')

# 6. Derive AES key using SHA-256
aes_key = SHA256.new(shared_x).digest()

aes_key.hex()

'bcb20b1a1e013c91621770289100fc7f925c945e43cdf055d2e52fe2c4dc4c31'

In [15]:
# Full communication example

import json
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES

# Alice begins the communication and generates the diffie hellman parameters

alice_ffdh = dh.generate_parameters(generator=2, key_size=2048)
alice_sk = parameters.generate_private_key()
alice_pk = alice_private_key.public_key()
alice_p = parameters.parameter_numbers().p
alice_g = parameters.parameter_numbers().g
alice_salt = get_random_bytes(16)

# Data that is sent over the internet to Bob:
datastring = json.dumps({
    'g': alice_g,
    'p': alice_p,
    'pky': alice_pk.public_numbers().y,
    'salt': alice_salt.hex(),
})


## NETWORK TRANSPORT of datastring


# Bob receives the data and unpacks it
handshake_from_alice = json.loads(datastring)
bob_g = handshake_from_alice['g']
bob_p = handshake_from_alice['p']
alice_pky = handshake_from_alice['pky']
bob_salt = bytes.fromhex(handshake_from_alice['salt'])


# Bob generates his own private key
bob_ffdh = dh.DHParameterNumbers(bob_p, bob_g).parameters()
bob_private_key = parameters.generate_private_key()
bob_public_key = bob_private_key.public_key()
# Bob calculates the shared secret
alice_pk = dh.DHPublicNumbers(alice_pky, dh.DHParameterNumbers(bob_p, bob_g)).public_key()
bob_shared_key = bob_private_key.exchange(alice_public_key)
print("Bob's shared key:", bob_shared_key.hex())

# Bob calculates encryption key 32 bytes long encryption key
bob_encryption_key = HKDF(
    algorithm = hashes.SHA256(), # Use the SHA-256 hash function, as it is fast and secure
    length = 32, # 256-bit key for AES-256
    salt = bob_salt, # Add salt to increase security even with repeated use of the same context and key
    info = b'cryptolab_custom_handshake_2025', # Application-specific context (different key for each use-case)
).derive(bob_shared_key)
print("Bob's encryption key:", alice_shared_key.hex())

# Bob sends his public key to Alice
datastring = json.dumps({
    'pky': bob_public_key.public_numbers().y,
})


## NETWORK TRANSPORT of datastring


# Alice receives the data and unpacks it
handshake_from_bob = json.loads(datastring)
bob_pky = handshake_from_bob['pky']
bob_public_key = dh.DHPublicNumbers(bob_pky, dh.DHParameterNumbers(alice_p, alice_g)).public_key()
# Alice calculates the shared secret
alice_shared_key = alice_private_key.exchange(bob_public_key)
print("Alice's shared key:", alice_shared_key.hex())
# Now both parties have the same shared secret!

assert alice_shared_key == bob_shared_key

# Alice calculates encryption key 32 bytes long encryption key
alice_encryption_key = HKDF(
    algorithm = hashes.SHA256(), # Use the SHA-256 hash function, as it is fast and secure
    length = 32, # 256-bit key for AES-256
    salt = alice_salt, # Add salt to increase security even with repeated use of the same context and key
    info = b'cryptolab_custom_handshake_2025', # Application-specific context (different key for each use-case)
).derive(alice_shared_key)
print("Alice's encryption key:", alice_shared_key.hex())

# HANDSHAKE COMPLETE!

# Alice encrypts a message using AES-GCM
alice_message = 'Hello Bob, this is Alice!'
print("Alice's message:", alice_message)
alice_message_bytes = alice_message.encode('utf-8')
alice_cipher = AES.new(alice_encryption_key, AES.MODE_GCM)
ciphertext, tag = alice_cipher.encrypt_and_digest(alice_message_bytes)
print("Ciphertext:", ciphertext.hex())
print("Tag:", tag.hex())

# Alice packs the ciphertext and tag into a JSON object
datastring = json.dumps({
    'ciphertext': ciphertext.hex(),
    'tag': tag.hex(),
    'nonce': alice_cipher.nonce.hex(),
})


## NETWORK TRANSPORT of ciphertext and tag


# Bob receives the ciphertext and tag
cipher_from_alice = json.loads(datastring)
alice_ciphertext = bytes.fromhex(cipher_from_alice['ciphertext'])
alice_tag = bytes.fromhex(cipher_from_alice['tag'])
alice_nonce = bytes.fromhex(cipher_from_alice['nonce'])

# Bob decrypts the message using AES-GCM
bob_cipher = AES.new(bob_encryption_key, AES.MODE_GCM, nonce=alice_nonce)
try:
    decrypted_messsage_bytes = bob_cipher.decrypt_and_verify(alice_ciphertext, alice_tag)
    decrypted_message = decrypted_messsage_bytes
    print("Bob's decrypted message:", decrypted_message.decode('utf-8'))
except ValueError:
    print("Decryption failed or message was tampered with!")

assert decrypted_message.decode('utf-8') == alice_message

Bob's shared key: 3e895141bfa99db45b52e8aa67079c695197b4e1f155963a82ec465dfb4345ebc972bf4ae074375e4c1cf4cd80cabff014d60d64a26efbf1b8f7da8b368d250ff45b6109e45427031db01768d3d655a368f513888619233d709899038f10344deb784ca3298861e023c6a16ebc262364c2791bdfe3bb00c9e3f6a3371ad081f51ac9c19da6f9c8851b5550a12b4611dfe40c53ea49b0128e880b894a8dd5db1308df0f254d71524fe6bed5d2d0201f6e5ed1cf0ec4623aeb29f54b2f679eddd26cdec6755c83be5cfd981bc8bf92bebc719be110339213b7cb0c1162a72b46e03dfa1773db1b74a3634d08945ae711d5b62f7f3f9b9b89ff5ffe59bfd8fdaa43
Bob's encryption key: 3d7f11768ebfd0f26ccf741eb9d12e12fe5d50bde97ac831db4fc374bd7ea2d8724b879dabb27ce2a716efbbcffc97a221cf4ea42802655c42fe5568a25517079fc3324dac4938af8d2b7f8f16de240d2a44768f7a9a99a6e98e74f4f0d8f58becd8d008289f8a168bf4e0d086540dde14bf0a1eb81cc7cd406577148aecb3c935c54f9b1b0328480244fc05323e2ba65a8a9823a821861d7d4aecf173d1464d0c82c5e1b12fbf2d86bcde9057717664abaaa21ac22a71f5c9c8cb649d4f2acf50ac647bdf9fe16e9a631d525be0b585cb5657032873fe0042f171ed4070f80